In [ ]:
from Bio import SeqIO
import subprocess
import os

human_file = "human.fa"
mouse_file = "mouse.fa"

#Create a BLAST database for mouse proteins
print("Creating BLAST database from mouse sequences...")
subprocess.run([
    "makeblastdb", 
    "-in", mouse_file, 
    "-dbtype", "prot", 
    "-out", "mouse_protein_db"
])

# Parse human protein sequences and run BLASTp for each sequence
output_file = "output_protein_blast.txt"
with open(output_file, "w") as out_file:
    for human_seq in SeqIO.parse(human_file, "fasta"):
        # Save the current human sequence to a temporary file
        temp_query_file = "temp_query.fa"
        SeqIO.write(human_seq, temp_query_file, "fasta")

        # Run BLASTp against the mouse protein database
        print(f"Running BLASTp for sequence {human_seq.id}...")
        result = subprocess.run(
            [
                "blastp", 
                "-query", temp_query_file, 
                "-db", "mouse_protein_db", 
                "-outfmt", "6 qseqid sseqid evalue bitscore"
            ],
            capture_output=True,
            text=True
        )

        # Write the BLASTp output to the final output file
        if result.stdout:
            out_file.write(result.stdout)
        else:
            print(f"No results found for sequence {human_seq.id}")

        # Remove the temporary query file after each search
        os.remove(temp_query_file)

print(f"BLAST results saved to {output_file}")


Creating BLAST database from mouse sequences...


Building a new DB, current time: 11/04/2024 15:36:27
New DB name:   /Users/katherine/Desktop/CSC649/HMW3/mouse_protein_db
New DB title:  mouse.fa
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/katherine/Desktop/CSC649/HMW3/mouse_protein_db
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 17489 sequences in 0.365083 seconds.


Running BLASTp for sequence STAU1...
Running BLASTp for sequence NRAS...
Running BLASTp for sequence HRAS...
Running BLASTp for sequence SLC30A10...
Running BLASTp for sequence CTSV...
Running BLASTp for sequence SFRS4...
Running BLASTp for sequence GABRG2...
Running BLASTp for sequence PCDH1...
Running BLASTp for sequence CAMK1G...
Running BLASTp for sequence ADCYAP1...
Running BLASTp for sequence PPP2R5D...
Running BLASTp for sequence MLLT3...
Running BLASTp for sequence VN1R3...
BLAST results saved to output_protein_blast.txt


(i) Choice of BLAST Program: blastp
Since the dataset consists of protein sequences, I use blastp, the BLAST program designed for protein-protein sequence comparisons. Protein BLAST is ideal for identifying homologous sequences at the amino acid level, which typically captures evolutionary relationships more effectively than nucleotide sequences alone. blastp is optimized to account for protein-specific alignment challenges.

(ii) Choice of Substitution Matrix: BLOSUM62
The default substitution matrix for blastp is typically BLOSUM62, which is widely used for protein alignment because it balances sensitivity and specificity, making it effective for finding homologs with around 62% sequence similarity. 
BLOSUM62 works well for finding both close and moderately distant homologs, covering a broad range of evolutionary distances.

(iii) Choice of Parameters

E-value (-evalue):
The E-value controls the stringency of the results. An E-value threshold of 1e-5 to 1e-3 is often used for finding homologous proteins while filtering out weak matches. Lowering the E-value threshold makes the search more stringent, reporting only highly significant matches, which is useful for identifying truly homologous proteins.

Output Format (-outfmt 6):
This format specifies a tabular output for easy parsing. It includes fields such as query ID, subject ID, E-value, and bitscore, providing concise and structured information for each alignment.

Database Parameters:
-db mouse_protein_db specifies the database created from mouse.fa. This ensures that each human protein is searched against the mouse database to identify homologous sequences.

Alignment Score (Bitscore):
The bitscore provides a normalized score for each alignment. Higher bitscores indicate better matches, making it easier to rank the homologs based on their similarity. Sorting by bitscore allows easy identification of the most homologous sequences.